## E1 - Deterministic Break Sampling in 2D
In experiment 1, we conducted deterministic folding of 1,000 predefined strings of proteins with lengths {5,10,15,…,95,200}, on a 2D lattice sized to twice the length of the maximum protein sequence. However, this second approach uses predetermined folding patterns, rather than random placement at each step, for each amino acid in the sequence. The predetermined directions were set to either {L, R, S}`*` for each amino acid, ensuring the path of the chain was decided before the simulation began.

The protein chain was extended by positioning each amino acid adjacent to the previous one according to its predetermined direction. This approach also avoids backward movements like experiment 1, but is more sensitive to collisions, since its path is predetermined and thus less flexible in avoiding adjacent neighbours, which might result in a collision. 

This deterministic chain-relative approach guarantees that the predetermined folding pattern is achieved for every simulation unless a collision is encountered along its path. The total count of amino acids was tracked as the chain unfolded on the grid. The folding process continued until all amino acids were placed on the grid or the predetermined adjacent placement location resulted in a collision.

`*` After generating proteins but before saving the directional data, the first two 'fixed' directions are removed for all proteins.

In [45]:
import numpy as np 
import pandas as pd
import time as time
np.set_printoptions(threshold=np.inf)

### Step 1 - Initializing Acids

This function `random_division` takes the total number of amino acids as input and randomly divides them into hydrophobic and polar amino acids, before creating a random 1D amino acid string. It ensures that there's at least one hydrophobic and one polar amino acid. You can adjust the total_amino_acids variable to the desired number of amino acids you want to divide.

In [74]:
total_amino_acids = 10  # Change this to a desired number, 10 is used for illustration purposes

def random_division(total_amino_acids):
    num_hydrophobic = np.random.randint(1, total_amino_acids)  # Ensure at least one H and P
    num_polar = total_amino_acids - num_hydrophobic
    # randomly shuffle the amino acids
    amino_acids = ['H'] * num_hydrophobic + ['P'] * num_polar
    np.random.shuffle(amino_acids)
    return num_hydrophobic, num_polar, amino_acids

num_hydrophobic, num_polar, amino_acids = random_division(total_amino_acids)
print("Number of Hydro acids:", num_hydrophobic)
print("Number of Polar acids:", num_polar)
print("Random Amino Acid String:", "".join(amino_acids))

Number of Hydro acids: 8
Number of Polar acids: 2
Random Amino Acid String: HHHHPHHPHH


The function `generate_random_directions` generates a random sequence of directions for amino acid placement. The first direction is 'Start', the second is 'straight', and the rest are randomly chosen from 'left', 'straight', and 'right'.

In [75]:
def generate_random_directions(total_amino_acids):

    # The first amino acid is 'Start' and the second is 'straight' by default
    directions = ['Start', 'straight']

    # For the remaining amino acids, choose randomly between 'left', 'straight', and 'right'
    for _ in range(2, total_amino_acids):
        directions.append(np.random.choice(['left', 'straight', 'right']))

    return directions

# Example usage
random_directions = generate_random_directions(total_amino_acids)
random_directions

['Start',
 'straight',
 'right',
 'right',
 'left',
 'straight',
 'straight',
 'left',
 'right',
 'right']

### Step 2 - Initializing Grid
The `initialize_grid` function generates a grid to place amino acids, with specified dimensions, and populates it with H and P amino acids, while ensuring they do not overlap and tracks their placement order. This allows for simulating the random arrangement of amino acids on a grid for various applications.

In [76]:
import numpy as np

def initialize_grid_directional(amino_acids, directions):
    amino_acids_copy = amino_acids.copy()
    total_amino_acids = len(amino_acids)
    grid_size = total_amino_acids * 2
    grid = np.full((grid_size, grid_size), fill_value='', dtype=object)
    amino_acid_order = []  # List to track the order of amino acids placed
    collision_count = 0  # Initialize collision counter

    # Start from the middle of the grid
    current_row, current_col = grid_size // 2, grid_size // 2

    # Initial movement is to the right (on the grid)
    current_orientation = 'right'

    # Movement vectors for the grid
    movement_vectors = {
        'up': (-1, 0),
        'down': (1, 0),
        'left': (0, -1),
        'right': (0, 1)
    }

    def get_new_orientation(current_orientation, turn):
        if current_orientation == 'up':
            return 'left' if turn == 'left' else 'right'
        elif current_orientation == 'down':
            return 'right' if turn == 'left' else 'left'
        elif current_orientation == 'left':
            return 'down' if turn == 'left' else 'up'
        elif current_orientation == 'right':
            return 'up' if turn == 'left' else 'down'

    for direction in directions:
        if direction == 'Start':
            amino_acid_type = amino_acids_copy.pop(0)
            grid[current_row, current_col] = amino_acid_type
            amino_acid_order.append((amino_acid_type, (current_row, current_col)))
        else:
            if direction != 'straight':
                current_orientation = get_new_orientation(current_orientation, direction)

            move = movement_vectors[current_orientation]
            next_row, next_col = current_row + move[0], current_col + move[1]

            if 0 <= next_row < grid_size and 0 <= next_col < grid_size:
                if grid[next_row, next_col] != '':
                    collision_count += 1  # Increment collision count if cell is occupied
                amino_acid_type = amino_acids_copy.pop(0)
                grid[next_row, next_col] = amino_acid_type
                amino_acid_order.append((amino_acid_type, (next_row, next_col)))
                current_row, current_col = next_row, next_col
            else:
                collision_count += 1  # Increment collision count if cell is outside the grid

            if not amino_acids_copy:
                break

    return grid, amino_acid_order, collision_count

grid, amino_acid_order, collision_count = initialize_grid_directional(amino_acids, random_directions)

In [77]:
print("Collisions:", collision_count)

Collisions: 0


In [57]:
print("Grid:", grid)

Grid: [['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' 'P' 'P' 'P' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' 'P' 'H' 'P' 'P' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' 'P' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' 'P' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '' '' '' '' '' ''

In [58]:
print("Amino Acid Order:", amino_acid_order)

Amino Acid Order: [('P', (10, 10)), ('P', (10, 11)), ('P', (10, 12)), ('P', (10, 13)), ('P', (9, 13)), ('P', (9, 12)), ('P', (9, 11)), ('H', (10, 11)), ('P', (11, 11)), ('P', (12, 11))]


In [59]:
def determine_directions(amino_acid_order):
    """
    Determines the direction of each amino acid placement relative to the previous one.
    Directions are 'left', 'straight', 'right', considering the orientation of the movement from the previous point.
    """
    directions = ['Start']  # First amino acid has no direction

    # Define movement vectors for easier comparison
    movement_vectors = {
        'up': (-1, 0),
        'down': (1, 0),
        'left': (0, -1),
        'right': (0, 1)
    }

    for i in range(1, len(amino_acid_order)):
        # Get the current and previous amino acid's row and column
        _, (current_row, current_col) = amino_acid_order[i]
        _, (prev_row, prev_col) = amino_acid_order[i - 1]

        # Determine the movement vector from the previous amino acid
        move_vector = (current_row - prev_row, current_col - prev_col)

        if i == 1:
            # For the second amino acid, we don't have a previous direction, so we set it as straight
            direction = 'straight'
        else:
            # Get the previous movement vector
            _, (prev_prev_row, prev_prev_col) = amino_acid_order[i - 2]
            prev_move_vector = (prev_row - prev_prev_row, prev_col - prev_prev_col)

            # Determine direction based on previous movement vector
            if prev_move_vector in [movement_vectors['up'], movement_vectors['down']]:
                # Moving vertically
                if move_vector == movement_vectors['left']:
                    direction = 'left' if prev_move_vector == movement_vectors['up'] else 'right'
                elif move_vector == movement_vectors['right']:
                    direction = 'right' if prev_move_vector == movement_vectors['up'] else 'left'
                else:
                    direction = 'straight'
            else:
                # Moving horizontally
                if move_vector == movement_vectors['up']:
                    direction = 'left' if prev_move_vector == movement_vectors['right'] else 'right'
                elif move_vector == movement_vectors['down']:
                    direction = 'right' if prev_move_vector == movement_vectors['right'] else 'left'
                else:
                    direction = 'straight'

        directions.append(direction)

    return directions

# Test the refined function with the provided example amino_acid_order
determine_directions(amino_acid_order)

['Start',
 'straight',
 'straight',
 'straight',
 'left',
 'left',
 'straight',
 'left',
 'straight',
 'straight']

In [60]:
def trim_empty_rows_and_columns(grid):
    # Find the indices of non-empty rows and columns
    non_empty_rows = np.any(grid != '', axis=1)
    non_empty_columns = np.any(grid != '', axis=0)

    # Use boolean indexing to extract non-empty rows and columns
    trimmed_grid = grid[non_empty_rows][:, non_empty_columns]

    return trimmed_grid

# Call the function to trim empty rows and columns
trimmed_grid = trim_empty_rows_and_columns(grid)

# Print the trimmed grid
print("Trimmed Grid:", '\n', trimmed_grid)

Trimmed Grid: 
 [['' 'P' 'P' 'P']
 ['P' 'H' 'P' 'P']
 ['' 'P' '' '']
 ['' 'P' '' '']]


### Step 3a - Calculate H-bonds
The `find_H_combinations_grid` function is used to~ identify and collect sets of coordinates representing adjacent 'H' amino acids in a grid. It iterates through the entire grid, checking each position for the presence of 'H' amino acids. If an 'H' amino acid is found, it examines neighboring positions (up, down, left, right) to identify adjacent 'H' amino acids. For each pair of adjacent 'H' amino acids, it creates a frozenset containing their coordinates (ensuring that the order of coordinates doesn't matter) and adds this frozenset to a set. This set stores all unique pairs of adjacent 'H' amino acids found in the grid. The function returns this set of adjacent 'H' amino acid pairs.

In [61]:
def find_H_pairs_grid(grid):
    adjacent_hydrophobic_amino_acids = set()  # Use a set to automatically remove duplicates

    # Iterate through the grid to find adjacent 'H' amino acids
    for row in range(grid.shape[0]):
        for col in range(grid.shape[1]):
            current_acid = grid[row, col]

            # Check if the current amino acid is 'H'
            if current_acid == 'H':
                # Check the neighboring positions (up, down, left, right) relative to the current position
                neighbors = [
                    (row - 1, col),
                    (row + 1, col),
                    (row, col - 1),
                    (row, col + 1)
                ]

                for neighbor_row, neighbor_col in neighbors:
                    # Check if the neighbor is within the grid bounds
                    if 0 <= neighbor_row < grid.shape[0] and 0 <= neighbor_col < grid.shape[1]:
                        neighbor_acid = grid[neighbor_row, neighbor_col]

                        # Check if the neighbor is also 'H'
                        if neighbor_acid == 'H':
                            # Use frozenset to ensure that the order of coordinates doesn't matter
                            amino_acid_pair = frozenset({(row, col), (neighbor_row, neighbor_col)})
                            adjacent_hydrophobic_amino_acids.add(amino_acid_pair)
                            
    return adjacent_hydrophobic_amino_acids

find_H_pairs_grid(grid)

set()

### Step 3b - Calculate H-bonds
The `find_H_combinations_order` function examines the order of amino acids and identifies adjacent 'H' amino acids. It does this by iterating through the amino acid order, checking pairs of consecutive amino acids for 'H' type, and recording these pairs as frozensets in a set to remove duplicates. This function helps identify adjacent 'H' amino acids in the sequence order, which is useful for analyzing the arrangement of amino acids.

In [62]:
def find_H_pairs_order(amino_acid_order):
    adjacent_hydrophobic_amino_acids = set()  # Use a set to automatically remove duplicates

    # Iterate through the amino acid order to find adjacent 'H' amino acids
    for i in range(len(amino_acid_order) - 1):
        current_acid, current_position = amino_acid_order[i]
        next_acid, next_position = amino_acid_order[i + 1]

        # Check if both current and next amino acids are 'H'
        if current_acid == 'H' and next_acid == 'H':
            # Use frozenset to ensure that the order of positions doesn't matter
            amino_acid_pair = frozenset({current_position, next_position})
            adjacent_hydrophobic_amino_acids.add(amino_acid_pair)

    return adjacent_hydrophobic_amino_acids

find_H_pairs_order(amino_acid_order)

set()

### Step 3c - Calculate H-bonds
The provided code calculates the H-combinations in two different ways and then compares them. It calculates the H-pairs in the grid using the `find_H_combinations_grid` function and the H-pairs in the amino acid order using the `find_H_combination_order` function. Then, it subtracts the H-combinations found in the amino acid order from those found in the grid. This comparison helps identify the **H-bonds** that are formed between adjacent 'H' amino acids in the grid but not in the given amino acid sequence order.

In [63]:
def find_H_bonds(grid, amino_acid_order):
    grid_h_pairs = find_H_pairs_grid(grid)
    order_h_pairs = find_H_pairs_order(amino_acid_order)
    return grid_h_pairs - order_h_pairs

H_bonds = find_H_bonds(grid, amino_acid_order)

In [64]:
print('Number of H-bonds:', len(H_bonds))

for bond in H_bonds:
    coordinates = [coord for coord in bond]
    print(coordinates)

Number of H-bonds: 0


### Step 4 - Creating Samples
The code generates N random protein grid configurations with a specified number of hydrophobic and polar amino acids using the `generate_random_samples` function. 

In [104]:
def generate_random_samples(N, total_amino_acids):
    random_samples = []

    for _ in range(N):
        num_hydrophobic, num_polar, amino_acids = random_division(total_amino_acids)
        amino_acids_copy = amino_acids.copy()
        initial_grid, amino_acid_order, collision_count = initialize_grid_directional(amino_acids, generate_random_directions(total_amino_acids))
        trimmed_grid = trim_empty_rows_and_columns(initial_grid)
        protein_dimensions = trimmed_grid.shape
        amino_acids_on_grid = np.count_nonzero(initial_grid != '')
        amino_acids_directions =  determine_directions(amino_acid_order)
        hbonds = len(find_H_bonds(initial_grid, amino_acid_order))
        hratio = hbonds / amino_acids_on_grid
        collision_count
        
        random_samples.append((num_hydrophobic,
                               num_polar,
                               amino_acids,
                               initial_grid,
                               amino_acids_on_grid,
                               amino_acid_order,
                               amino_acids_directions,
                               trimmed_grid, 
                               protein_dimensions,
                               hbonds,
                               hratio,
                               collision_count,
                              ))

    return random_samples

# Example usage
generate_random_samples(1, total_amino_acids)

[(4,
  6,
  ['H', 'P', 'H', 'P', 'P', 'H', 'P', 'H', 'P', 'P'],
  array([['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
          '', '', '', ''],
         ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
          '', '', '', ''],
         ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
          '', '', '', ''],
         ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
          '', '', '', ''],
         ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
          '', '', '', ''],
         ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
          '', '', '', ''],
         ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
          '', '', '', ''],
         ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
          '', '', '', ''],
         ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
          '', '', '', ''],
         ['', '', '', '', '

In [105]:
def timing_samples(num_samples, amino_acid_lengths):
    total_times = {}

    for length in amino_acid_lengths:
        start_time = time.time()
        
        _ = store_samples_in_dataframe(num_samples, [length])  # We call the original function here
        
        end_time = time.time()
        
        total_times[length] = end_time - start_time
        print(f"Time taken to create and process {num_samples} samples for amino acid length {length}: {total_times[length]:.2f} seconds")

    return total_times

In [106]:
def store_samples_in_dataframe(num_samples, amino_acid_lengths):
    
    data = {
        "Amino Acid Length": [],
        "Num Hydrophobic": [],
        "Num Polar": [],
        "1D protein": [],
        "2D protein": [],
        "Amino Acids on Grid": [],  
        "Trimmed 2D protein": [],
        "Shape 2D protein": [],
        "Amino Acid Order": [],
        "Amino Acid Direction": [],
        "H-Bonds": [],
        "H-Ratio": [],
        "Collisions": [],
    }

    for length in amino_acid_lengths:
        
        random_samples = generate_random_samples(num_samples, length)
        for sample in random_samples:
            
            start_time = time.time()  # Start time before producing the sample
            num_hydrophobic, num_polar, amino_acids, initial_grid, amino_acids_on_grid, amino_acid_order, determine_directions, trimmed_grid, protein_dimensions, hbonds, hratio, collision_count= sample
            end_time = time.time()  # End time after producing the sample
            
            data["Amino Acid Length"].append(length)
            data["Num Hydrophobic"].append(num_hydrophobic)
            data["Num Polar"].append(num_polar)
            data["1D protein"].append(amino_acids)
            data["2D protein"].append(initial_grid)
            data["Amino Acids on Grid"].append(amino_acids_on_grid)
            data["Trimmed 2D protein"].append(trimmed_grid)
            data["Shape 2D protein"].append(protein_dimensions)
            data["Amino Acid Order"].append(amino_acid_order)
            data["Amino Acid Direction"].append(determine_directions),
            data["H-Bonds"].append(hbonds)
            data['H-Ratio'].append(hratio)
            data["Collisions"].append(collision_count)

    df = pd.DataFrame(data)
    return df

In [126]:
%%time
# Generate an experiment with 1000 samples for amino length [5, 10, 15 ..., 200]

num_samples = 1000
amino_acid_lengths = [length for length in range(5, 201, 5)]
time_data = timing_samples(num_samples, amino_acid_lengths)
HP200 = store_samples_in_dataframe(num_samples, amino_acid_lengths)

Time taken to create and process 1000 samples for amino acid length 5: 0.15 seconds
Time taken to create and process 1000 samples for amino acid length 10: 0.25 seconds
Time taken to create and process 1000 samples for amino acid length 15: 0.39 seconds
Time taken to create and process 1000 samples for amino acid length 20: 0.57 seconds
Time taken to create and process 1000 samples for amino acid length 25: 0.77 seconds
Time taken to create and process 1000 samples for amino acid length 30: 1.00 seconds
Time taken to create and process 1000 samples for amino acid length 35: 1.30 seconds
Time taken to create and process 1000 samples for amino acid length 40: 1.57 seconds
Time taken to create and process 1000 samples for amino acid length 45: 1.89 seconds
Time taken to create and process 1000 samples for amino acid length 50: 2.44 seconds
Time taken to create and process 1000 samples for amino acid length 55: 2.62 seconds
Time taken to create and process 1000 samples for amino acid lengt

In [127]:
def shorten_direction(direction_list):
    # Create a dictionary mapping full words to their respective initials
    mapping = {'Start': 'S', 'start': 'S', 'middle': 'M', 'right': 'R', 'left': 'L', 'straight': 'S'}
    # Use the mapping to convert each word to its initial and join them into a string
    return ''.join(mapping[direction] for direction in direction_list)

HP200['Full Acid Path'] = HP200['Amino Acid Direction'].apply(lambda x: shorten_direction(x))

In [128]:
# Remove the first two characters from each string (Start and straight)
HP200['Shortened Acid Path'] = HP200['Full Acid Path'].apply(lambda x: x[2:])

In [129]:
# Selecting the specified columns to create smaller dataframe
HP200_small = HP200[['Amino Acid Length', 'Amino Acids on Grid', 'Shortened Acid Path', 'Collisions']]
HP200_small.tail()

,Amino Acid Length,Amino Acids on Grid,Shortened Acid Path,Collisions
39995,200,113,LRLLSRSLLRLSRSSLRLLRSLLLLLRRLRRSRRLRLSRSSLRRRS...,87
39996,200,118,SRSLRRSLLRRSLRLRSLRLLSLRRRLSLRSRRLRLRSSRRLRLSS...,82
39997,200,117,SRLRSSRSRRLSLRSRSSLSSLLRRSLLLRRRRLSRLSRRRSLRSR...,83
39998,200,127,SRRSLLSRRLSSLRRRLLSSSLLSLLSRSLRRSRLRSRLLRLRSRR...,73
39999,200,106,RRRLRRLSLRLSLLSLSSLRSRRSRSSRSLRLRLSSLRSRRSLSRS...,94


In [132]:
HP200_small.to_csv('../Paper/HP200 small E1.csv', index=False)